In [18]:
# all imports up here, rerun this block when adding
import pandas as pd
import numpy as np
import cv2 as cv

Read Data

In [3]:
#assumes dataset in folder
wines   = pd.read_csv("XWines_Full_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
ratings = pd.read_csv("XWines_Full_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
len(wines), len(ratings)

print(ratings)
print(wines)

          RatingID   UserID  WineID Vintage  Rating                 Date
0                1  1604441  136103    1950     4.0  2019-10-14 11:20:52
1                2  1291483  136103    1950     5.0  2019-11-28 03:36:33
2                3  1070605  104036    1950     5.0  2017-12-28 10:15:55
3                4  1080181  144864    1950     5.0  2016-06-23 02:16:22
4                5  1834379  111430    1950     5.0  2021-05-16 17:58:14
...            ...      ...     ...     ...     ...                  ...
21013531  21013532  2015383  113302    N.V.     3.0  2019-02-16 14:15:48
21013532  21013533  1868739  111440    N.V.     2.0  2018-09-30 16:47:05
21013533  21013534  1402947  142467    N.V.     3.0  2021-01-29 19:21:14
21013534  21013535  1360350  111440    N.V.     4.0  2021-07-26 14:02:14
21013535  21013536  1192603  111393    N.V.     5.0  2016-11-17 04:48:43

[21013536 rows x 6 columns]
        WineID                           WineName       Type  \
0       100001                 

In [20]:
def read():
    ##assumes dataset in folder
    wines   = pd.read_csv("XWines_Full_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
    ratings = pd.read_csv("XWines_Full_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
    return wines,ratings

Hardcoded Decision Tree with Type, Country and Body

In [32]:

wine_type = input("Enter the type of wine (e.g., red, white, rosé, sparkling): ").lower()
wine_country = input("Enter the country of the wine (e.g., Brazil, New Zealand): ").lower()
wine_body = input("Enter the body of the wine(e.g. ligth-bodied, medium-bodied, full-bodied): ").lower()

filtered_wines = wines.copy()

filtered_wines = wines[(wines['Type'].str.lower() == wine_type) 
                       & (wines['Country'].str.lower() == wine_country)
                       & (wines['Body'].str.lower() == wine_body)]

if filtered_wines.empty:
    filtered_wines = wines[(wines['Type'].str.lower() == wine_type)
                           & (wines['Country'].str.lower() == wine_country)]
    if not filtered_wines.empty:
        print(f"No '{wine_type}' wines in country '{wine_country}' found with body '{wine_body}'")
        print(f"Showing all '{wine_type}' wines from country '{wine_country}' instead.")
    else:
        filtered_wines = wines[wines['Type'].str.lower() == wine_type]
        if not filtered_wines.empty:
            print(f"No '{wine_type}' wines in country '{wine_country}'. Showing all '{wine_type}'instead")
        else:
            print(f"No wines found for the type '{wine_type}'.")
else:
    print(f"Showing '{wine_type}' wines from '{wine_country}'.")

if not filtered_wines.empty:
    print(filtered_wines)
else:
    print("No wines found that match your preferences.")

Showing 'red' wines from 'brazil'.
       WineID                                  WineName Type  \
2      100003                        Cabernet Sauvignon  Red   
4      100005           Maison de Ville Cabernet-Merlot  Red   
5      100006                Reserva Cabernet Sauvignon  Red   
7      100008               Paradoxo Cabernet Sauvignon  Red   
8      100009         Seleção Cabernet Sauvignon-Merlot  Red   
...       ...                                       ...  ...   
1553   101555                            Reserva Tannat  Red   
1558   101560         Brazilian Soul Cabernet Sauvignon  Red   
1560   101562           Serra Gaucha Cabernet Sauvignon  Red   
53507  153559                 Vinho Tinto De Mesa Suave  Red   
66913  167017  Edicao Especial para Primitivo Búzios-RJ  Red   

                           Elaborate                                Grapes  \
2                      Varietal/100%                ['Cabernet Sauvignon']   
4      Assemblage/Bordeaux Red Blend    

Creating Wine Features Function

In [12]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def create_wine_features(wine_data):
    """
    Create and combine wine features (textual, categorical, and numerical) from the wine data.
    
    Args:
    - wine_data: DataFrame containing wine feature data.
    
    Returns:
    - wine_features: Sparse matrix of combined wine features (without weights).
    """
    
    # Make a copy of the wine data to avoid modifying the original DataFrame
    wine_data = wine_data.copy()

    # Preprocessing 'Type' and 'Country' columns (convert list to space-separated string)
    wine_data['Type'] = wine_data['Type'].apply(lambda x: ' '.join(eval(x)))
    wine_data['Country'] = wine_data['Country'].apply(lambda x: ' '.join(eval(x)))

    # TF-IDF vectorization for textual features (Type, Country, Body)
    tfidf = TfidfVectorizer()
    wine_data_tfidf_type = tfidf.fit_transform(wine_data['Type'])
    wine_data_tfidf_country = tfidf.fit_transform(wine_data['Country'])
    wine_data_tfidf_body = tfidf.fit_transform(wine_data['Body'])

    # One-hot encoding for categorical features
    one_hot = OneHotEncoder()
    categorical_features = ['Type', 'Body', 'Acidity', 'Country', 'RegionName', 'WineryName']
    wine_categorical_onehot = one_hot.fit_transform(wine_data[categorical_features])

    # Scale the numerical feature 'ABV'
    scaler = StandardScaler()
    wine_abv_scaled = scaler.fit_transform(wine_data[['ABV']])

    # Combine all features into a single sparse matrix
    wine_features = sp.hstack([
        wine_data_tfidf_type, 
        wine_data_tfidf_country, 
        wine_data_tfidf_body,
        wine_categorical_onehot, 
        wine_abv_scaled
    ]).tocsr()

    return wine_features

In [33]:

# Filter ratings based on the wines you are interested in
filtered_reviews = ratings[ratings['WineID'].isin(filtered_wines['WineID'])]

# Merge filtered_wines and filtered_reviews on 'WineID'
merged_df = pd.merge(filtered_wines, filtered_reviews, on='WineID', how='inner')

# Identify columns that belong to the wines and ratings DataFrames
wine_columns = ['WineID', 'Type', 'Country', 'Body']  
rating_columns = ['RatingID', 'WineID', 'UserID', 'Rating']  

# Split the merged DataFrame back into wines and ratings
filtered_wines = merged_df[wine_columns].drop_duplicates()  # Drop duplicates to get unique wine data
filtered_reviews = merged_df[rating_columns]

# Print to confirm filtering is correct
print("Filtered Wines:")
print(filtered_wines.head(10))
print("Filtered Reviews:")
print(filtered_reviews.head(10))


Filtered Wines:
      WineID Type Country         Body
0     100003  Red  Brazil  Full-bodied
62    100005  Red  Brazil  Full-bodied
134   100006  Red  Brazil  Full-bodied
1971  100008  Red  Brazil  Full-bodied
2395  100009  Red  Brazil  Full-bodied
4366  100018  Red  Brazil  Full-bodied
4558  100019  Red  Brazil  Full-bodied
4731  100020  Red  Brazil  Full-bodied
5251  100021  Red  Brazil  Full-bodied
6980  100023  Red  Brazil  Full-bodied
Filtered Reviews:
   RatingID  WineID   UserID  Rating
0   1741828  100003  1144814     4.0
1   1780869  100003  1259528     2.5
2   3327100  100003  2033195     4.0
3   3731950  100003  1964905     3.0
4   4116770  100003  1195131     3.0
5   4117322  100003  1155871     3.5
6   4145653  100003  1207297     3.0
7   4157416  100003  1183536     4.0
8   6054500  100003  1668029     4.0
9   6410901  100003  1731345     5.0


In [44]:
import random
import numpy as np
from sklearn.model_selection import train_test_split
from CB import ContentBasedRecommender
from CB_similarities import CBRecommender

# Split into train and test
train_ratings, test_ratings = train_test_split(filtered_reviews, test_size=0.2, random_state=42)

# Create an instance of the ContentBasedRecommender
recommender = ContentBasedRecommender(train_ratings, filtered_wines)  # Pass train_ratings to initialize

# Fit the model using the training ratings
recommender.fit(train_ratings)

# Choose a random user ID from the test set
test_user_id = np.random.choice(test_ratings['UserID'].unique())

# Check if the user has rated any wines in the test set
if test_user_id not in train_ratings['UserID'].values:
    print(f"User ID {test_user_id} has not rated any wines in the training set.")
else:
    # Get recommendations for the test user ID
    recommendations = recommender.recommend(test_user_id, top_n=5)
    print("Recommendations for user {}: {}".format(test_user_id, recommendations))



ValueError: b'gamma < 0'